In [1]:
import numpy as np
import torch
from torch import Tensor
from torch import tensor
from torch import nn
from matplotlib import pyplot as plt
import matplotlib as mpl
import torchvision
import json

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x00000272C8CC9A90>>
Traceback (most recent call last):
  File "c:\Users\minhh\Downloads\Projects\sundayai\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 790, in _clean_thread_parent_frames
    active_threads = {thread.ident for thread in threading.enumerate()}
  File "C:\Users\minhh\AppData\Local\Programs\Python\Python313\Lib\threading.py", line 1477, in enumerate
    def enumerate():
KeyboardInterrupt: 


In [13]:
device = 'cuda'

In [14]:
train_data = torchvision.datasets.FashionMNIST(
    root="data",
    train=True,
    transform=torchvision.transforms.ToTensor(),
    download=True
)
test_data = torchvision.datasets.FashionMNIST(
    root="data",
    train=False,
    transform=torchvision.transforms.ToTensor(),
    download=True,
)
train_data_loader = torch.utils.data.DataLoader(train_data,batch_size=4)

In [15]:
def save(name, obj):
    with open(name, "w") as f:
        json.dump(obj, f)
    f.close()

In [16]:
class MnistClassifier(nn.Module):
    def __init__(self,in_channels,out_features,device = 'cuda',dtype= float):
        super().__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels,out_channels=32,kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64,kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.fc1 = nn.Linear(in_features=64*6*6,out_features=600)
        self.drop = nn.Dropout2d(0.25)
        self.fc2 = nn.Linear(in_features=600,out_features=120)
        self.fc3 = nn.Linear(in_features=120,out_features=10)
    def forward(self,x:torch.Tensor):
        x = self.block1(x)
        x = self.block2(x)
        x = x.view(x.size(0),-1)
        x = self.fc1(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x
    def sample_loss_landscape(self, dataloader, criterion, d1: int, d2: int, space=torch.linspace(-1, 1, 50), device='cuda'):
        with torch.no_grad():
            # Use first 10 samples
            for x_batch, y_batch in dataloader:
                x_batch, y_batch = x_batch[:10].to(device), y_batch[:10].to(device)
                break

            # Flatten parameters
            original_params = torch.nn.utils.parameters_to_vector(self.parameters()).detach()
            num_params = original_params.numel()

            # Create unit vectors in specified directions
            dir1 = torch.zeros_like(original_params)
            dir2 = torch.zeros_like(original_params)
            dir1[d1] = 1.0
            dir2[d2] = 1.0

            # Project original parameters onto these directions
            alpha_0 = original_params[d1].item()
            beta_0 = original_params[d2].item()
            outputs = self.forward(x_batch)
            loss_0 = criterion(outputs, y_batch)
            # Loss grid
            loss_grid = torch.zeros((len(space), len(space))).cpu()

            for i, alpha in enumerate(space):
                for j, beta in enumerate(space):
                    new_params = original_params + alpha * dir1 + beta * dir2
                    torch.nn.utils.vector_to_parameters(new_params, self.parameters())

                    outputs = self.forward(x_batch)
                    loss = criterion(outputs, y_batch)
                    loss_grid[i, j] = loss.item()

            # Restore original parameters
            torch.nn.utils.vector_to_parameters(original_params, self.parameters())

        return loss_grid, torch.tensor([alpha_0, beta_0,loss_0])




In [21]:
loss_fn = nn.CrossEntropyLoss()
epochs = 2

In [22]:
def save_numpy(fname, data):
    np.save(fname, data)

In [26]:
model = MnistClassifier(1,10).to(device=device)
optimizer = torch.optim.Adam(model.parameters())
loss_grids = None
trajectories = None
global_count = 0

In [27]:

for epoch in range(epochs):
    for img, label in train_data_loader:
        img = img.to(dtype=torch.float32).to(device=device)
        label = torch.nn.functional.one_hot(label,num_classes=10).to(dtype=torch.float32).to(device=device)
        label = label.to(device=device)
        pred = model(img)
        optimizer.zero_grad()
        loss = loss_fn(pred,label)
        loss.backward()
        optimizer.step()
        print(loss.mean().item())

c:\Users\minhh\Downloads\Projects\sundayai\.venv\Lib\site-packages\torch\nn\functional.py:1538: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


2.5670032501220703
4.747742652893066
4.18553352355957
1.2012219429016113
10.914383888244629
7.158888339996338
2.5709750652313232
2.366027355194092
5.475251197814941
4.190726280212402
4.326183795928955
6.749291896820068
1.2335877418518066
4.961446285247803
2.6390039920806885
1.142754077911377
2.472163200378418
3.348344326019287
5.711630344390869
3.994662284851074
1.0631331205368042
0.3037871718406677
1.619123935699463
3.3055102825164795
4.104753017425537
3.486759662628174
3.556688070297241
3.9209306240081787
1.451930046081543
1.019301414489746
3.3632326126098633
2.351527690887451
2.846580743789673
1.196214199066162
2.0987472534179688
3.097036838531494
0.49373847246170044
1.3945293426513672
0.9692013263702393
1.8521239757537842
1.3255281448364258
1.7036495208740234
1.5496153831481934
1.8167027235031128
1.357950210571289
2.1725378036499023
2.0486690998077393
2.2034826278686523
0.29569342732429504
3.2497189044952393
4.17141056060791
2.3901073932647705
0.24368683993816376
1.4225215911865234

SGD 8867/10000  loss: 0.133
Adam 8823/10000 loss: 0.055

In [28]:
correct = 0
total = len(test_data)
test_loader = torch.utils.data.DataLoader(test_data,batch_size=1)
model = model.to('cpu')
for img, label in test_loader:
    pred = model(img).argmax().item()
    if(pred == label):
        correct+=1
print(f"{correct} {total}")

8823 10000
